In [7]:
#Name: Azeem Waqar
#ID: 21I-0679
#Section: CS-B

In [10]:
import random

class TimetableScheduler:
    def __init__(self, courses, professors, sections, rooms, course_types):
        self.courses = courses
        self.professors = professors
        self.sections = sections
        self.rooms = rooms
        self.course_types = course_types
        self.num_days = 5
        self.morning_slots = 6
        self.afternoon_slots = 3
        self.morning_session_start = 8.5
        self.afternoon_session_start = 14.5
        self.slot_duration = 1.25
        self.break_duration = 0.25
        self.professor_course_limit = 3
        self.section_course_limit = 5
        self.theory_session_slots = 2
        self.lab_session_slots = 1

    def initialize_population(self, population_size):
        population = []
        for _ in range(population_size):
            chromosome = self.generate_chromosome()
            population.append(chromosome)
        return population

    def generate_chromosome(self):
        chromosome = []
        # Generate chromosome for each course
        for course in self.courses:
            course_chromosome = []
            # Add details for first lecture
            # Course, Theory/Lab, Section, Section-Strength, Professor, First-lecture-day,
            # First-lecture-timeslot, First-lecture-room, First-lecture-room-size, Second-lecture-day,
            # Second-lecture-timeslot, Second-lecture-room, Second-lecture-room-size
            course_chromosome.append(course)
            course_type = random.choice(self.course_types)
            course_chromosome.append(course_type)  # Theory or Lab
            # Add section details
            section = random.choice(self.sections)
            course_chromosome.extend(section)
            # Add professor details
            professor = random.choice(self.professors)
            course_chromosome.append(professor)
            # Add day and timeslot details
            day, time_slot = self.generate_timeslot()
            course_chromosome.extend([day, time_slot])
            # Add room details
            room, room_size = self.select_room(course_type)
            course_chromosome.extend([room, room_size])
            # Add details for second lecture (if applicable)
            if course_type == "Theory":
                day, time_slot = self.generate_timeslot()
                course_chromosome.extend([day, time_slot])
                room, room_size = self.select_room("Theory")
                course_chromosome.extend([room, room_size])
            else:
                # Lab lectures should be conducted in two consecutive slots
                next_day, next_time_slot = self.generate_timeslot(increment=1)
                course_chromosome.extend([next_day, next_time_slot, room, room_size])
            chromosome.append(course_chromosome)
        return chromosome

    def generate_timeslot(self, increment=0):
        day = random.randint(1, self.num_days)
        if increment:
            time_slot = random.randint(1, self.morning_slots - 1)
        else:
            time_slot = random.randint(1, self.morning_slots)
        return day, time_slot

    def select_room(self, course_type):
        if course_type == "Theory":
            room = random.choice([room for room in self.rooms if room[1] >= 60])
        else:
            room = random.choice([room for room in self.rooms if room[1] >= 120])
        return room

    def fitness_function(self, chromosome):
        conflicts = 0
        professor_course_count = {}
        section_course_count = {}
        

        # Initialize dictionaries to track assigned professors, rooms, and sections
        assigned_professors = {}
        assigned_rooms = {}
        assigned_sections = {}

        for i, course_chromosome in enumerate(chromosome):
            if len(course_chromosome) != 13:
                continue

            course, course_type, section, section_strength, professor, \
            first_day, first_time_slot, first_room, first_room_size, \
            second_day, second_time_slot, second_room, second_room_size = course_chromosome

            # Constraint 1: Classroom size
            if first_room_size < section_strength or second_room_size < section_strength:
                conflicts += 1

            # Constraint 2 and 4: Professor and room assignment
            for assigned in [assigned_professors, assigned_rooms]:
                for key, assigned_courses in assigned.items():
                    for assigned_course in assigned_courses:
                        if assigned_course[5] == first_day and assigned_course[6] == first_time_slot:
                            conflicts += 1
                        if assigned_course[9] == second_day and assigned_course[10] == second_time_slot:
                            conflicts += 1

            # Constraint 3: Section assignment
            for assigned_course in assigned_sections.get(section, []):
                if assigned_course[5] == first_day and assigned_course[6] == first_time_slot:
                    conflicts += 1
                if assigned_course[9] == second_day and assigned_course[10] == second_time_slot:
                    conflicts += 1

            # Update counts for Constraint 5 and 6
            professor_course_count[professor] = professor_course_count.get(professor, 0) + 1
            section_course_count[section] = section_course_count.get(section, 0) + 1

            # Update assigned dictionaries
            assigned_professors.setdefault(professor, []).append(course_chromosome)
            assigned_rooms.setdefault(first_room, []).append(course_chromosome)
            assigned_sections.setdefault(section, []).append(course_chromosome)

        # Constraint 5: Professor course limit
        for courses in professor_course_count.values():
            if courses > 3:
                conflicts += courses - 3

        # Constraint 6: Section course limit
        for courses in section_course_count.values():
            if courses > 5:
                conflicts += courses - 5

        # Constraint 7: Lecture days
        for course_chromosome in chromosome:
            if course_chromosome[1] == 'Theory'and \
            course_chromosome[5] == course_chromosome[9] or \
              abs(course_chromosome[5] - course_chromosome[9]) == 1:
                conflicts += 1

        # Constraint 8: Lab lectures
        for course_chromosome in chromosome:
            if course_chromosome[1] == 'Lab' and \
              (course_chromosome[5] != course_chromosome[9] or \
                abs(course_chromosome[6] - course_chromosome[10]) != 1):
                conflicts += 1

        return -conflicts


    def crossover(self, parent1, parent2):
        # Select a random crossover point
        crossover_point = random.randint(1, len(parent1) - 1)

        # Create offspring by combining parents' chromosomes
        offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
        offspring2 = parent2[:crossover_point] + parent1[crossover_point:]

        return offspring1, offspring2

    def mutate(self, chromosome, mutation_rate=0.9):
        mutated_chromosome = chromosome.copy()

        # Iterate through each gene in the chromosome
        for i in range(len(mutated_chromosome)):
            # Randomly decide whether to mutate this gene
            if random.random() < mutation_rate:
                # Mutate the gene
                gene = mutated_chromosome[i]
                if i % 13 == 1:  # Mutate course type (Theory/Lab)
                    gene[i] = "Theory" if gene == "Lab" else "Lab"
                elif i % 13 == 5:  # Mutate first lecture day
                    gene[i] = random.randint(1, self.num_days)
                elif i % 13 == 6:  # Mutate first lecture timeslot
                    gene[i] = random.randint(1, self.morning_slots)
                elif i % 13 == 7 or i % 13 == 10:  # Mutate first and second lecture room
                    room, _ = self.select_room(gene)
                    gene[i] = room
                elif i % 13 == 8 or i % 13 == 11:  # Mutate first and second lecture room size
                    room, room_size = self.select_room(chromosome[i - 1])
                    gene[i] = room_size
                elif i % 13 == 9:  # Mutate second lecture day
                    gene[i] = random.randint(1, self.num_days)
                elif i % 13 == 12:  # Mutate second lecture timeslot
                    gene[i] = random.randint(1, self.morning_slots)

                mutated_chromosome[i] = gene

        return mutated_chromosome

    def evolve(self, population, generations):
        i=0
        #generation = 0
        for generation in range(generations):
            # Calculate fitness for each chromosome in the population
            fitness_scores = [self.fitness_function(chromosome) for chromosome in population]

            # Select parents for crossover
            # Using tournament selection
            tournament = random.sample(range(len(population)), 5)  # Tournament size of 5
            tournament_fitness = [(i, self.fitness_function(population[i])) for i in tournament]
            tournament_fitness.sort(key=lambda x: x[1], reverse=True)  # Sort by fitness value in descending order
            top_indices = [t[0] for t in tournament_fitness[:2]]  # Select top 2 indices
            sp = top_indices

            # Perform crossover to create offspring
            offspring_population = []
            parent1 = population[sp[0]]
            parent2 = population[sp[1]]
            offspring1, offspring2 = self.crossover(parent1, parent2)

            # Perform mutation on the offspring population
            offspring1=self.mutate(offspring1)
            offspring2=self.mutate(offspring2)
            # Replace the old population with the mutated population

            population.append(offspring1)
            population.append(offspring2)

            population = sorted(population, key=lambda i: self.fitness_function(i), reverse = True)

            population=population[:10]

            # Print the best fitness score in each generation
            best_fitness = max(fitness_scores)
            print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")
            print("Best Chromosome: ", population[0])

        # Return the best chromosome after all generations
        best_chromosome_index = fitness_scores.index(max(fitness_scores))
        return population[best_chromosome_index]

# Example usage
courses = ["Math", "Physics", "Biology", "Chemistry"]
professors = ["Prof. A", "Prof. B", "Prof. C"]
sections = [("A", 50), ("B", 60), ("C", 70)]
rooms = [("Room1", 60), ("Room2", 120), ("Room3", 60)]
course_types = ["Theory", "Lab"]

scheduler = TimetableScheduler(courses, professors, sections, rooms, course_types)
population = scheduler.initialize_population(population_size=100)

pop = scheduler.evolve(population, generations=100)

Generation 1, Best Fitness: -1
Best Chromosome:  [['Math', 'Theory', 'B', 60, 'Prof. B', 3, 3, 'Room3', 60, 1, 5, 'Room2', 120], ['Physics', 'Lab', 'A', 50, 'Prof. B', 5, 3, 'Room2', 120, 1, 4, 'Room2', 120], ['Biology', 'Theory', 'A', 50, 'Prof. B', 5, 6, 'Room1', 60, 2, 6, 'Room3', 60], ['Chemistry', 'Theory', 'A', 50, 'Prof. C', 5, 1, 'Room2', 120, 2, 5, 'Room3', 60]]
Generation 2, Best Fitness: -1
Best Chromosome:  [['Math', 'Theory', 'B', 60, 'Prof. B', 3, 3, 'Room3', 60, 1, 5, 'Room2', 120], ['Physics', 'Lab', 'A', 50, 'Prof. B', 5, 3, 'Room2', 120, 1, 4, 'Room2', 120], ['Biology', 'Theory', 'A', 50, 'Prof. B', 5, 6, 'Room1', 60, 2, 6, 'Room3', 60], ['Chemistry', 'Theory', 'A', 50, 'Prof. C', 5, 1, 'Room2', 120, 2, 5, 'Room3', 60]]
Generation 3, Best Fitness: -1
Best Chromosome:  [['Math', 'Theory', 'B', 60, 'Prof. B', 3, 3, 'Room3', 60, 1, 5, 'Room2', 120], ['Physics', 'Lab', 'A', 50, 'Prof. B', 5, 3, 'Room2', 120, 1, 4, 'Room2', 120], ['Biology', 'Theory', 'A', 50, 'Prof. B', 5